In [ ]:
import sys
sys.path += ["../src"]
import utils
from glob import glob
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib.pyplot import subplots as sbp 
from importlib import reload
import jl_vae
import jl_nflows_geo_coordinates_2 as nfg
from jl_nflows_geo_coordinates import load_nf as load_dict

from _51_abm_functions import cod_prov_abbrv_df

# Global Spatial Autocorrelation
from spatial_autocorrelation import get_moransI, moransI_scatterplot, hypothesis_testing
# Local Spatial Autocorrelation
from spatial_autocorrelation import get_localMoransI, LISA_scatterplot
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.preprocessing import OneHotEncoder
from jl_synthetic_pop_all_provinces import add_cat_features
from ipfn import ipfn
import config


In [80]:
import sys
sys.path += ["../src"]
import utils
from glob import glob
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib.pyplot import subplots as sbp 
from importlib import reload
import jl_vae
import jl_nflows_geo_coordinates_2 as nfg
from jl_nflows_geo_coordinates import load_nf as load_dict

from _51_abm_functions import cod_prov_abbrv_df

# Global Spatial Autocorrelation
from spatial_autocorrelation import get_moransI, moransI_scatterplot, hypothesis_testing
# Local Spatial Autocorrelation
from spatial_autocorrelation import get_localMoransI, LISA_scatterplot
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.preprocessing import OneHotEncoder
from jl_synthetic_pop_all_provinces import add_cat_features
from ipfn import ipfn
import config

def transform_bins(df, vars):
    df1 = df.copy()
    for var in vars:
        df1[var + "_bin"] =  pd.qcut(df1[var], q = 5, labels = False, duplicates = "drop").astype(int)
        df1.drop(columns = [var], inplace = True)
    return df1

def transform_bins_to_real(sample_df, df_real, var):
    bins_values = pd.qcut(df_real[var], q = 5, labels = False, duplicates = "drop", retbins = True)[1]
    np.random.seed(2)
    sample_df[var] = [np.random.uniform(low = bins_values[u], high = bins_values[u + 1]) for u in sample_df[var + "_bin"]]
    sample_df.drop(columns = [var + "_bin"], inplace = True)
    return sample_df

    


In [ ]:

if True:
    # geo_dict = jl_vae.load_geo_data()
    date = "250717"

    for _, (prov, cod_prov) in cod_prov_abbrv_df.sort_values("prov_abbrv").iterrows():
        print(prov)
        real_pops = jl_vae.path_pop_synth + f"pop_samples/pop_real_with_hedonic_price"
        df_real_ = pd.read_csv(real_pops + f"/pop_real_full_250110{prov}.csv", index_col = 0)
        df_real_ = add_cat_features(df_real_)
        df_real = df_real_.assign(log_price = lambda x: x["price_corrected"], CAP = lambda x: x["CAP"].astype(str))

        np.random.seed(2)
        # ran_xy = np.random.uniform(low = df_real["x"].min(), high = df_real["x"].max(), size = 1000000), np.random.uniform(low = df_real["y"].min(), high = df_real["y"].max(), size = 1000000)

        # df_locations = utils.spatial_matching_ABM(pd.DataFrame({"x": ran_xy[0], "y": ran_xy[1]}), geo_dict["hydro_risk"], geo_dict["census"], 
        #                                           geo_dict["omi_og"], geo_dict["cap"]).query("prov_abbrv == @prov")
        
        df_real = df_real_.assign(log_price = lambda x: x["price_corrected"], CAP = lambda x: x["CAP"].astype(str))
        df_real = df_real[['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
            'flag_multi_floor', 
            'log_mq', 'log_price',
            'flag_air_conditioning_Missing', 
            # 'x', 'y', 
            'CAP',
            'energy_class', 
            'COD_CAT',
            'anno_costruzione']]
        
        df_sample_list = []

        for cap in df_real["CAP"].unique():
            df_cap = df_real.query("CAP == @cap").dropna()
            df_cap = transform_bins(df_cap, ["log_mq", "log_price"])
            N_cap = len(df_cap)
            df_counts = df_cap.value_counts().reset_index()
            cap_locs = df_locations.query("CAP == @cap").sample(n = N_cap, random_state = 2)
            sample_df_cap = df_counts.sample(n = 10 * N_cap, weights = df_counts["count"],
                                         random_state = 2, replace = True).drop(columns = ["count"])

            sample_df_cap[["x", "y"]] = cap_locs[["GEO_LONGITUDINE_BENE_ROUNDED", "GEO_LATITUDINE_BENE_ROUNDED"]]
            sample_df_cap["CAP"] = cap
            sample_df_cap = transform_bins_to_real(sample_df_cap, df_real, "log_mq")
            sample_df_cap = transform_bins_to_real(sample_df_cap, df_real, "log_price")

            df_sample_list.append(sample_df_cap)
        sample_df = pd.concat(df_sample_list)
        sample_df.to_csv(jl_vae.path_pop_synth + f"ipf_samples/pop_sample_{prov}_{date}.csv")

AG


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [82]:
df_cap

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,log_mq,log_price,flag_air_conditioning_Missing,CAP,energy_class,COD_CAT,anno_costruzione
573,0.0,0.0,0.0,0.0,5.780744,45753.761922,1.0,92028,Low_energy_class,A_01_07_08,1985_2005


In [63]:
df_ = pd.DataFrame({"a": [1,2,3], "b": [4,5,6], "c": [45,55,65], "d": [42,25,16], "e": [14,15,61]})
df_ = pd.DataFrame({"a": [1], "b": [4], "c": [45], "d": [42], "e": [14]})

In [64]:
pd.qcut(df_["a"], q = 5, labels = False)

ValueError: Bin edges must be unique: array([1., 1., 1., 1., 1., 1.]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [53]:
transform_bins(df_real.dropna(), ["log_mq", "log_price"])["log_price_bin"].sort_values()

573    0
715    0
967    0
975    0
194    0
      ..
953    4
709    4
718    4
679    4
293    4
Name: log_price_bin, Length: 295, dtype: int64

In [15]:
transform_bins(df_real.query("CAP == @cap"), ["log_price", "log_mq"])

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,flag_air_conditioning_Missing,CAP,energy_class,COD_CAT,anno_costruzione,log_price_bin,log_mq_bin
0,1.0,0.0,0.0,0.0,0.0,92024,NaN,A02,1985_2005,4,2
1,0.0,0.0,0.0,0.0,0.0,92024,NaN,A02,1985_2005,3,2
12,1.0,0.0,0.0,0.0,0.0,92024,NaN,A02,1500_1965,4,4
29,0.0,0.0,0.0,0.0,0.0,92024,NaN,A02,2005_2025,2,2
43,0.0,0.0,0.0,0.0,0.0,92024,NaN,A_04_05,1500_1965,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1128,0.0,0.0,0.0,0.0,0.0,92024,NaN,A03,1965_1985,0,0
1142,0.0,0.0,0.0,0.0,0.0,92024,NaN,A03,1965_1985,1,1
1153,1.0,0.0,0.0,0.0,0.0,92024,Low_energy_class,A02,1965_1985,2,2
1167,0.0,1.0,0.0,0.0,0.0,92024,NaN,A03,1965_1985,4,4


In [70]:

if True:
    geo_dict = jl_vae.load_geo_data()
    date = "250717"

    for _, (prov, cod_prov) in cod_prov_abbrv_df.sort_values("prov_abbrv").iterrows():
        print(prov)
        real_pops = jl_vae.path_pop_synth + f"pop_samples/pop_real_with_hedonic_price"
        df_real_ = pd.read_csv(real_pops + f"/pop_real_full_250110{prov}.csv", index_col = 0)
        df_real_ = add_cat_features(df_real_)
        df_real = df_real_.assign(log_price = lambda x: x["price_corrected"], CAP = lambda x: x["CAP"].astype(str))

        np.random.seed(2)
        # ran_xy = np.random.uniform(low = df_real["x"].min(), high = df_real["x"].max(), size = 1000000), np.random.uniform(low = df_real["y"].min(), high = df_real["y"].max(), size = 1000000)

        # df_locations = utils.spatial_matching_ABM(pd.DataFrame({"x": ran_xy[0], "y": ran_xy[1]}), geo_dict["hydro_risk"], geo_dict["census"], 
        #                                           geo_dict["omi_og"], geo_dict["cap"]).query("prov_abbrv == @prov")
        
        df_real = df_real_.assign(log_price = lambda x: x["price_corrected"], CAP = lambda x: x["CAP"].astype(str))
        df_real = df_real[['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
            'flag_multi_floor', 
            'log_mq', 'log_price',
            'flag_air_conditioning_Missing', 
            # 'x', 'y', 
            'CAP',
            'energy_class', 
            'COD_CAT',
            'anno_costruzione']]
        
        
        df_sample_list = []

        for cap in df_real["CAP"].unique():
            df_cap = df_real.query("CAP == @cap").dropna()
            df_cap = transform_bins(df_cap, ["log_mq", "log_price"])
            N_cap = len(df_cap)
            df_counts = df_cap.value_counts().reset_index()
            cap_locs = df_locations.query("CAP == @cap").sample(n = N_cap)
            sample_df_cap = df_counts.sample(n = 10 * N_cap, weights = df_counts["count"],
                                         random_state = 2, replace = True).drop(columns = ["count"])

            sample_df_cap[["x", "y"]] = cap_locs[["GEO_LONGITUDINE_BENE_ROUNDED", "GEO_LATITUDINE_BENE_ROUNDED"]]
            sample_df_cap["CAP"] = cap
            sample_df_cap = transform_bins_to_real(sample_df_cap, df_real, "log_mq")
            sample_df_cap = transform_bins_to_real(sample_df_cap, df_real, "log_price")

            df_sample_list.append(sample_df_cap)
        sample_df = pd.concat(df_sample_list)
        sample_df.to_csv(jl_vae.path_pop_synth + f"ipf_samples/pop_sample_{prov}_{date}.csv")

AG


ValueError: Bin edges must be unique: array([4.7791233 , 4.7791233 , 4.7791233 , 4.78735494, 4.80381822,
       4.8202815 ]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [79]:
pd.qcut(df_cap["log_mq"], q = 5, duplicates = "drop")

55      (4.804, 4.82]
411    (4.778, 4.787]
937    (4.778, 4.787]
Name: log_mq, dtype: category
Categories (3, interval[float64, right]): [(4.778, 4.787] < (4.787, 4.804] < (4.804, 4.82]]

In [35]:
# pd.qcut(df_real["log_price"], q = 5, labels = False).astype(int)



In [44]:
transform_bins_to_real(sample_df_cap, df_real.dropna(), "log_mq")

KeyError: 'log_mq_bin'

In [47]:
var = "log_mq"
bins_values = pd.qcut(df_real[var], q = 5, labels = False, retbins = True)[1]
sample_df_cap[var] = [np.random.uniform(low = bins_values[u], high = bins_values[u + 1]) for u in sample_df_cap[var + "_bin"].astype(int)]
sample_df_cap.drop(columns = [var + "_bin"], inplace = True)
    

KeyError: 'log_mq_bin'

In [8]:
[np.random.uniform(low = bins_values[int(u)], high = bins_values[u + 1]) for u in sample_df_cap[var + "_bin"].astype(int)]

[67999.47766220749,
 88891.75394802286,
 62459.95164240125,
 68789.42968938522,
 68393.42658232212,
 80286.13850330842,
 82631.34233792883,
 71606.522599609,
 67119.62765749023,
 75957.96000142403,
 71913.31978432756,
 65092.52055887565,
 68508.51122383654,
 64307.09664160719,
 87232.41203047583,
 82697.67683308704,
 77588.84837333686,
 68432.2075609575,
 80157.29896698843,
 69525.98534329115,
 61784.57230163988,
 73495.23346346036,
 75305.25752937575,
 93695.67276601953,
 62554.86810705537,
 91132.95201142,
 79667.33179728662,
 108511.31393805917,
 76234.68507864405,
 90114.6440915314,
 200741.30399436352,
 76826.52414839804,
 104594.03629704841,
 63880.12913526062,
 66316.47272832706,
 63391.276567642795,
 88591.09086780681,
 101263.8058725016,
 85575.0033541955,
 102485.79472937899,
 96600.1700299079,
 70971.14293172833,
 76197.6521962065,
 84511.88724390013,
 94469.09810037856,
 69479.5271068393,
 63702.00886751701,
 88379.8952420558,
 75204.65537349011,
 84251.25867580585,
 62200.

In [7]:
sample_df_cap[var + "_bin"].astype(int)

10    1
0     2
13    1
10    1
10    1
     ..
1     1
21    2
16    3
3     1
10    1
Name: log_price_bin, Length: 1220, dtype: int64